In [ ]:
import os.path
import time

import torch
torch.cuda.empty_cache()

In [2]:
from src.efficentnet_train import data_load,models_train,visualization,utils,generate_dataset,evaluation
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader
from src.efficentfacenet import face_descriptor
from torchvision import transforms
import pandas as pd



In [2]:
dataset_path = "../dataset/preprocessed"
model_weights_path="../model_weights/training"
train_data_save_path="../training log"
batch_size=8
stop_n_layers=5

In [3]:
train_history=pd.read_csv(train_data_save_path+"/train_data.csv",index_col=False)

In [5]:
train_history

,Train Loss,no train rows,Test Loss,No test rows,Time taken (M),Notes,Date,Time
0,0.280173,100000,0.342320,25000,65.2,remove sigmoid from classifier + input normali...,15/05/2022,23:26:00
1,0.989897,100000,1.000320,25000,60.1,remove sigmoid from classifier + input normali...,16/05/2022,00:27:00
2,1.000000,100000,1.000320,25000,60.2,remove sigmoid from classifier + input normali...,16/05/2022,01:27:00
3,0.173325,50000,0.284327,10000,63.0,remove sigmoid from classifier + input normali...,16/05/2022,19:28:00
4,0.243919,50000,0.281481,10000,60.0,remove sigmoid from classifier + input normali...,16/05/2022,20:28:00
5,0.210516,50000,0.238477,10000,59.9,remove sigmoid from classifier + input normali...,16/05/2022,21:28:00
6,0.122000,50000,0.259760,10000,62.9,remove sigmoid from classifier + input normali...,16/05/2022,22:38:00
7,0.197626,50000,0.260942,10000,60.1,remove sigmoid from classifier + input normali...,16/05/2022,23:38:00
8,0.168668,50000,0.216822,10000,60.2,remove sigmoid from classifier + input normali...,17/05/2022,00:38:00
9,0.097149,25000,0.268254,5000,31.3,Add sigmoid to classifier + input normalization,17/05/2022,14:29:00


In [1]:
model = face_descriptor.FaceDescriptorModel(download_weights=True, version="efficientnet_b1")



NameError: name 'face_descriptor' is not defined

In [8]:
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [5]:
train_transform = transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 240)),transforms.RandomHorizontalFlip(),transforms.RandomRotation(10),transforms.RandomAutocontrast(),data_load.Normalize()])
test_transform=transforms.Compose([transforms.ToTensor(), transforms.Resize((240, 240)),data_load.Normalize()])

In [7]:
model

EfficientNet(
  (features): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): SiLU(inplace=True)
    )
    (1): Sequential(
      (0): MBConv(
        (block): Sequential(
          (0): ConvNormActivation(
            (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
            (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): SiLU(inplace=True)
          )
          (1): SqueezeExcitation(
            (avgpool): AdaptiveAvgPool2d(output_size=1)
            (fc1): Conv2d(32, 8, kernel_size=(1, 1), stride=(1, 1))
            (fc2): Conv2d(8, 32, kernel_size=(1, 1), stride=(1, 1))
            (activation): SiLU(inplace=True)
            (scale_activation): Sigmoid()
          )
          (2): ConvNormActivation(
 

In [6]:
train_dataset = data_load.FacesDataset(f"{dataset_path}/train", 5000, train_transform,True)
test_dataset=data_load.FacesDataset(f"{dataset_path}/test", 1000, test_transform,True)


 100.0 % of the folders processed img dict loaded in 0.26 m
 100.0 % of the folders processed img dict loaded in 0.05 m


In [11]:
train_dataset.no_of_rows=10000
test_dataset.no_of_rows=2500

In [7]:
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [33]:
a_batch, p_batch, n_batch = next(iter(train_dataset))

In [34]:
a_batch.shape

torch.Size([3, 240, 240])

In [44]:
# for i in range(30):
#     _,ax=plt.subplots(1,3)
#     ax[0].set_title("Anchor")
#     ax[0].imshow(a[i])
#
#     ax[1].set_title("Positive")
#     ax[1].imshow(p[i])
#
#     ax[2].set_title("Negative")
#     ax[2].imshow(n[i])

In [12]:
import torch
import time
x=torch.tensor([1,2])
x.unsqueeze(1)


tensor([[1],
        [2]])

In [ ]:
ts=time.time()
a_batch, p_batch, n_batch = next(iter(train_loader))
print(time.time()-ts)
model.eval()
with torch.no_grad():
    for i in range(batch_size):

        fig, ax = plt.subplots(1, 3)

        # anchor_vector=model(a_batch[i].unsqueeze(0)).detach()[0]
        # positive_vector=model(p_batch[i].unsqueeze(0)).detach()[0]
        # negative_vector=model(n_batch[i].unsqueeze(0)).detach()[0]

        ax[0].set_title("anchor")
        ax[0].imshow(a_batch[i].numpy().transpose([1,2,0]))


        ax[1].set_title("positive")
        ax[1].imshow(p_batch[i].numpy().transpose([1,2,0]))



        ax[2].set_title("negative")
        ax[2].imshow(n_batch[i].numpy().transpose([1,2,0]))




        # print(f"anchor vs Positive = {utils.euclidean_distance(anchor_vector.numpy(),positive_vector.numpy())}")
        # print(f"anchor vs negative = {utils.euclidean_distance(anchor_vector.numpy(),negative_vector.numpy())}")

In [8]:
for i in range (len(model.features)):
      for parm in model.features[i].parameters():
            parm.requires_grad=True

In [19]:
for i in range(stop_n_layers):
  for parm in model.features[i].parameters():
    parm.requires_grad=True


In [9]:
epochs=15
learning_rate=0.001
train_notes="Classifier -->(1280,256,128)+ full train , randomHorizontal flip,random Rotation , randomAutoContrast,effnetb1 , learnrate =0.0001"


In [11]:
model.load_local_weights("../model_weights/training/05_20 18_23 Train_(0.164040) Test_(0.167041).pt",True)

In [12]:
train_losses=models_train.triplet_loss_train(model,epochs=1000,learn_rate=0.0001,train_loader=train_loader,test_loader=test_loader,cuda=True,weight_saving_path=model_weights_path,epoch_data_saving_path=train_data_save_path,notes=train_notes)

Testing before training ...
 epoch 1 [==========] time remaining = 0.0 Avg Train_Loss=0.16056560822
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.16681166669
 epoch 1 train_loss =0.16056545398235322 test_loss=0.16681104350090026
new minimum test loss 0.166811  achieved, model weights saved 
!!!Warning Overfitting!!!
 epoch 2 [==========] time remaining = 0.0 Avg Train_Loss=0.16182660915
 Testing  [==========] time remaining = 0.0 Avg Test_Loss=0.18263180268
 epoch 2 train_loss =0.16182647721767426 test_loss=0.18263150191307068
!!!Warning Overfitting!!!
 epoch 3 [==........] time remaining = 5.139126 Avg Train_Loss=0.160807

KeyboardInterrupt: 

In [8]:

saving_path=model_weights_path,epoch_data_saving_path=train_data_save_path,notes=train_notes)

NameError: name 'train_notes' is not defined

In [14]:
model.load_local_weights(model_weights_path+"/05_19 05_11 Train_(0.203699) Test_(0.367653).pt",cuda_weights=True)

In [20]:
features_dict=data_load.get_pic_features_dict(f"{dataset_path}/test",model,test_transform,cuda=True)

 data processed [==========] time remaing=0.006

In [7]:
import numpy as np
type(np.array([[1,2,3]]))

numpy.ndarray

In [21]:
test_data_pd=pd.DataFrame(generate_dataset.generate_testing_data_set_frame(dataset_path+"/test",True))

In [26]:
evaluation.model_test(features_dict,test_data_pd,results_path="",threshold=3.9)

 Testing  [=====.....] time remaining = 0.037384 Accuracy =87.013

KeyboardInterrupt: 

In [8]:
evaluation.euclidean_distance(np.array([5,2,3]),[1,5,8])

7.0710678118654755